In [127]:
from z3 import *
import numpy as np
class f2Poly:
    def __init__(self, terms=None):
        self.terms = set() if terms is None else terms
    def __add__(self, other):
        return f2Poly(self.terms ^ other.terms)
    def __mul__(self, other):
        terms1 = self.terms
        # 情况1：other 是数字（int）
        if isinstance(other, int):
            scalar = other % 2  # 模 2 到 F₂
            if scalar == 0:
                return f2Poly(set())  # 0 * p = 0
            else:
                return self           # 1 * p = p
        # 情况2：other 是 f2Poly
        terms2 = other.terms
        if not terms1 or not terms2:
            return f2Poly(set())  # 0 * anything = 0
        result = set()
        _add = result.add
        _remove = result.remove
        _in = result.__contains__
        for (i1, j1) in terms1:
            for (i2, j2) in terms2:
                term = (i1 + i2, j1 + j2)
                if _in(term):
                    _remove(term)
                else:
                    _add(term)
        return f2Poly(result)
    def __pow__(self, n):
        if n == 0: return f2Poly({(0, 0)})
        if len(self.terms) == 0:  # 零多项式
            if n > 0: return f2Poly(set())
            else: raise ValueError("0 的负幂次无定义")
        if len(self.terms) == 1:  # 单项式
            (i, j) = next(iter(self.terms))
            return f2Poly({(i * n, j * n)})
        # 多项式：快速幂
        if n < 0: raise ValueError("不可逆")
        elif n == 1: return self
        elif n % 2 == 0: return (self * self) ** (n // 2)
        else: return self * (self ** (n - 1))
    def __eq__(self, other):
        if not isinstance(other, f2Poly):
            return False
        return self.terms == other.terms
    def deg(self):
        x_powers = set()
        y_powers = set()
        if self.terms==set():
            x_powers.add(0)
            y_powers.add(0)
        else:
            for i, j in self.terms:
                x_powers.add(i)
                y_powers.add(j)
        return x_powers, y_powers
    def __repr__(self):
        if not self.terms:
            return "0"
        return " + ".join(
            f"x^{i}y^{j}" if i and j else
            f"x^{i}" if i else
            f"y^{j}" if j else
            "1"
            for i, j in self.terms
        )
def mod2(matrix):
    return np.mod(matrix, 2)
def gf2_rref(matrix):
    matrix = mod2(matrix)
    rows, cols = matrix.shape
    row, col = 0, 0
    while row < rows and col < cols:
        if matrix[row, col] == 0:
            for r in range(row + 1, rows):
                if matrix[r, col] == 1:
                    matrix[[row, r]] = matrix[[r, row]]
                    break
        if matrix[row, col] == 1:
            for r in range(rows):
                if r != row and matrix[r, col] == 1:
                    matrix[r] = mod2(matrix[r] + matrix[row])
            row += 1
        col += 1
    return matrix
def antimap(poly):
    inverted_terms = {( -i, -j ) for (i, j) in poly.terms}
    return f2Poly(inverted_terms)
def commute(a, b):
    # 预提取输入
    a0, a1, a2, a3 = a[0][0], a[1][0], a[2][0], a[3][0]
    b0, b1, b2, b3 = b[0][0], b[1][0], b[2][0], b[3][0]
    _antimap = antimap
    _f2Poly = f2Poly
    result = _f2Poly()
    result += _antimap(a0) * b2
    result += _antimap(a1) * b3
    result += _antimap(a2) * b0
    result += _antimap(a3) * b1
    return result
def excimap(*A, oper):
    result = []
    for a in A:
        excitation = commute(a, oper)
        result.append(excitation)
    return result  # 返回 [F2Poly, F2Poly, ...]，就是一个行向量
def tdmap(f, m):
    shifts = [(i, j) for j in range(-m, m + 1) for i in range(-m, m + 1)]
    f_terms_list = [poly.terms for poly in f]
    result = [None] * len(shifts)
    for idx, (i, j) in enumerate(shifts):
        shifted_f = []
        for terms in f_terms_list:
            shifted_terms = {(i + di, j + dj) for (di, dj) in terms}
            shifted_f.append(f2Poly(shifted_terms))
        result[idx] = shifted_f
    return result
def trunmap(f,k):
    ff=np.zeros((2*k+1)**2,dtype=int)
    for (i, j) in f.terms:
        ff[i+k+(j+k)*(2*k+1)]=1
    return ff
def kdeg(f,m):
    a=[]
    b=[]
    for i in range(len(tdmap(f,m)[0])):
        if tdmap(f,m)[0][i]!=f2Poly() and tdmap(f,m)[0][i]!=f2Poly({(0,0)}):
            a.append(max(max(max(tdmap(f,m)[0][i].deg()[0]),-min(tdmap(f,m)[0][i].deg()[0])),
                     max(max(tdmap(f,m)[0][i].deg()[1]),-min(tdmap(f,m)[0][i].deg()[1]))))
    for i in range(len(tdmap(f,m)[-1])):
        if tdmap(f,m)[-1][i]!=f2Poly() and tdmap(f,m)[-1][i]!=f2Poly({(0,0)}):
            b.append(max(max(max(tdmap(f,m)[-1][i].deg()[0]),-min(tdmap(f,m)[-1][i].deg()[0])),
                     max(max(tdmap(f,m)[-1][i].deg()[1]),-min(tdmap(f,m)[-1][i].deg()[1]))))
    return max(max(a),max(b))
def invtrunmap(f,k):
    ff=f2Poly()
    for i in range(len(f)):
        if f[i]!=0:
            a=i%(2*k+1)-k
            b=i//(2*k+1)-k
            ff=ff+f2Poly({(a,b)})
    return ff
def m1(*A,m):
    m1=[]
    m1.extend(tdmap(excimap(*A,oper=x1),m))
    m1.extend(tdmap(excimap(*A,oper=x2),m))
    m1.extend(tdmap(excimap(*A,oper=z1),m))
    m1.extend(tdmap(excimap(*A,oper=z2),m))
    k=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
    k=max(k,m+2)
    a=[]
    for i in range(len(m1)):
        b=[]
        for j in range(len(m1[i])):
            b.extend(trunmap(m1[i][j],k))
        a.append(b)
    return np.array(a)
def m2(*A,m):
    stabdag=[]
    for a in A:
        stabdag.append(np.array(a).T[0])
    for i in range(len(stabdag)):
        for j in range(len(stabdag[i])):
            stabdag[i][j] = antimap(stabdag[i][j])
    M2=[]
    for i in range(len(stabdag)):
        M2.extend(tdmap(stabdag[i],m))
    k=0
    for i in range(len(stabdag)):
       k=max(k,kdeg(stabdag[i],m))
    k=max(k,m+1)
    a=[]
    for i in range(len(M2)):
        b=[]
        for j in range(len(M2[i])):
            b.extend(trunmap(M2[i][j],k))
        a.append(b)
    return np.array(a)
def to(*A,m,mdag):
    m1i=np.hstack((m1(*A,m=m),np.eye(len(m1(*A,m=m)))))
    gem1i=mod2(gf2_rref(m1i))
    gem1,gei=np.hsplit(gem1i,[m1(*A,m=m).shape[1]])
    zeroidx=[]
    for i in range(len(gem1)):
        if all(x==0 for x in gem1[i]):
            zeroidx.append(i)
    nonzerorow,zerorow=np.vsplit(gei,[zeroidx[0]])
    zerorowx,zerorowz=np.hsplit(zerorow,[int(zerorow.shape[1]/2)])
    zerorowx1,zerorowx2=np.hsplit(zerorowx,[int(zerorowx.shape[1]/2)])
    zerorowz1,zerorowz2=np.hsplit(zerorowz,[int(zerorowz.shape[1]/2)])
    o=[]
    for i in range(len(zerorow)):
        oo=[]
        oo.append([invtrunmap(zerorowx1[i],m)])
        oo.append([invtrunmap(zerorowx2[i],m)])
        oo.append([invtrunmap(zerorowz1[i],m)])
        oo.append([invtrunmap(zerorowz2[i],m)])
        o.append(oo)
    o=np.array(o)
    odag=[]
    kk=max(int((np.sqrt(m2(*A,m=mdag).shape[1]/len(s1))-1)/2),mdag+1)
    for i in range(len(o)):
        oodag=[]
        for j in range(len(o[i])):
            oodag.extend(trunmap(antimap(o[i][j][0]),kk))
        odag.append(oodag)
    odag=np.array(odag)
    m2rank=np.sum(np.any(mod2(gf2_rref(m2(*A,m=mdag))), axis=1))
    n=0
    z=0
    for i in range(len(odag)):
        s=np.vstack((m2(*A,m=mdag),odag[i]))
        n=n+1
        if np.sum(np.any(mod2(gf2_rref(s)), axis=1))==m2rank:
            z=z+1
    if n==z:
        return True
    else:
        return False
def m3(*A,m,n,va):
    m1=[]
    m1.extend(tdmap(excimap(*A,oper=x1),m))
    m1.extend(tdmap(excimap(*A,oper=x2),m))
    m1.extend(tdmap(excimap(*A,oper=z1),m))
    m1.extend(tdmap(excimap(*A,oper=z2),m))
    k=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
    if va=='x':
        for i in range(len(A)):
            VV=[]
            for j in range(len(A)):
                if i==j:
                    VV.append(f2Poly({(0,0),(n,0)}))
                else:
                    VV.append(f2Poly())
            m1.extend(tdmap(VV,m))
            k=max(kdeg(VV,m),k)
    elif va=='y':
        for i in range(len(A)):
            VV=[]
            for j in range(len(A)):
                if i==j:
                    VV.append(f2Poly({(0,0),(0,-n)}))
                else:
                    VV.append(f2Poly())
            m1.extend(tdmap(VV,m))
            k=max(kdeg(VV,m),k)
    k=max(k,m+2)
    a=[]
    for i in range(len(m1)):
        b=[]
        for j in range(len(m1[i])):
            b.extend(trunmap(m1[i][j],k))
        a.append(b)
    return np.array(a)
def anyon(*A,m,N,va):
    a=[]
    zero=f2Poly()
    e=[]
    for i in range(len(A)):
        e.append(zero)
    ee=[]
    for i in range(4):
        ee.append(zero)
    for n in range(N):
        m3i=np.hstack((m3(*A,m=m,n=n,va=va),np.eye(len(m3(*A,m=m,n=n,va=va)))))
        gem3i=mod2(gf2_rref(m3i))
        gem3,gei=np.hsplit(gem3i,[m3(*A,m=m,n=n,va=va).shape[1]])
        zeroidx=[]
        for i in range(len(gem3)):
            if all(x==0 for x in gem3[i]):
                zeroidx.append(i)
        nonzerogeii,zerogeii=np.vsplit(gei,[zeroidx[0]])
        geie,geiv=np.hsplit(zerogeii,[(2*m+1)**2*4])
        geivv=np.array(np.hsplit(geiv,len(A)))
        p=[]
        p.append(e)
        for i in range(len(geivv[0])):
            p.append([invtrunmap(geivv[0][i],m),invtrunmap(geivv[1][i],m)])
        u=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
        u=max(u,m+2)
        for i in range(len(p)):
            for j in range(i+1,len(p)):
                s=[]
                for k in range(len(p[i])):
                    s.append(p[i][k]+p[j][k])
                c=[]
                for l in range(len(s)):
                    c.extend(trunmap(s[l],u))
                c=np.array(c)
                m1v=np.vstack((m1(*A,m=m),[c]))
                if np.sum(np.any(mod2(gf2_rref(m1v)), axis=1))==np.sum(np.any(mod2(gf2_rref(m1(*A,m=m))), axis=1)):
                    p[j]=e
        p = [x for x in p if x != e]
        o=[]
        for i in range(len(p)):
            t=[]
            for j in range(len(p[i])):
                t.extend(trunmap(p[i][j],u))
            o.append(t)
        m1vv=m1(*A,m=m)
        for i in range(len(o)):
            rank=np.sum(np.any(mod2(gf2_rref(m1vv))))
            m1vv=np.vstack((m1vv,o[i]))
            if np.sum(np.any(mod2(gf2_rref(m1vv)), axis=1))==rank:
                p[i]=e
        any=np.array([x for x in p if x != e])
        a.append(len(any))
    for i in range(len(a)):
        if a[i]==max(a):
            b=i
            break
    n=b
    m3i=np.hstack((m3(*A,m=m,n=n,va=va),np.eye(len(m3(*A,m=m,n=n,va=va)))))
    gem3i=mod2(gf2_rref(m3i))
    gem3,gei=np.hsplit(gem3i,[m3(*A,m=m,n=n,va=va).shape[1]])
    zeroidx=[]
    for i in range(len(gem3)):
        if all(x==0 for x in gem3[i]):
            zeroidx.append(i)
    nonzerogeii,zerogeii=np.vsplit(gei,[zeroidx[0]])
    geie,geiv=np.hsplit(zerogeii,[(2*m+1)**2*4])
    geivv=np.array(np.hsplit(geiv,len(A)))
    geiee=np.array(np.hsplit(geie,4))
    p=[]
    exci=[]
    exci.append(ee)
    f=len(geivv[0])
    g=len(geivv)
    p.append(e)
    for i in range(f):
        pp=[]
        for j in range(g):
            pp.append(invtrunmap(geivv[j][i],m))
        p.append(pp)
    f=len(geiee[0])
    g=len(geiee)
    for i in range(f):
        excit=[]
        for j in range(g):
            excit.append(invtrunmap(geiee[j][i],m))
        exci.append(excit)
    u=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
    u=max(u,m+2)
    for i in range(len(p)):
        for j in range(i+1,len(p)):
            s=[]
            for k in range(len(p[i])):
                s.append(p[i][k]+p[j][k])
            c=[]
            for l in range(len(s)):
                c.extend(trunmap(s[l],u))
            c=np.array(c)
            m1v=np.vstack((m1(*A,m=m),[c]))
            if np.sum(np.any(mod2(gf2_rref(m1v)), axis=1))==np.sum(np.any(mod2(gf2_rref(m1(*A,m=m))), axis=1)):
                p[j]=e
                exci[j]=ee
    p = [x for x in p if x != e]
    exci= [x for x in exci if x != ee]
    o=[]
    for i in range(len(p)):
        t=[]
        for j in range(len(p[i])):
            t.extend(trunmap(p[i][j],u))
        o.append(t)
    m1vv=m1(*A,m=m)
    for i in range(len(o)):
        rank=np.sum(np.any(mod2(gf2_rref(m1vv))))
        m1vv=np.vstack((m1vv,o[i]))
        if np.sum(np.any(mod2(gf2_rref(m1vv)), axis=1))==rank:
            p[i]=e
            exci[i]=ee
    any=np.array([x for x in p if x != e])
    exci=[x for x in exci if x != ee]
    k=np.sum(np.any(mod2(gf2_rref(m1vv)), axis=1))-np.sum(np.any(mod2(gf2_rref(m1(*A,m=m))), axis=1))
    exc=[]
    for i in range(len(exci)):
        excc=[]
        for j in range(len(exci[i])):
            excc.append([exci[i][j]])
        exc.append(excc)
    exc=np.array(exc)
    return any,exc,n,k/2
def stringoper(*A,m,N,va):
    if va=='x':
        x=anyon(*A,m=m,N=N,va='x')
        return x[1]
    if va=='y':
        x=anyon(s1,s2,m=m,N=N,va='x')
        y=anyon(s1,s2,m=m,N=N,va='y')
        m=m+max(x[2],y[2])
        nx=x[2]
        ny=-y[2]
        coff=f2Poly({(nx,0),(0,ny)})
        s=[]
        for i in range(len(x[0])):
            vv=[x*coff for x in x[0][i]]
            u=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
            u=max(u,m+2)
            vvf=[]
            for j in range(len(vv)):
                vvf.extend(trunmap(vv[j],u))
            m1v=np.vstack((m1(*A,m=m),[vvf]))
            m1vi=np.hstack((m1v,np.eye(len(m1v))))
            gem1vi=mod2(gf2_rref(m1vi))
            gem1v,gei=np.hsplit(gem1vi,[len(m1v[0])])
            zeroidx=[]
            for l in range(len(gem1v)):
                if all(x==0 for x in gem1v[l]):
                    zeroidx.append(l)
            nonzerogei,zerogei=np.vsplit(gei,[zeroidx[0]])
            zerogeie,zerogeiv=np.hsplit(zerogei,[len(zerogei[0])-1])
            for k in range(len(zerogeiv)):
                if zerogeiv[k][0]==1:
                    break
            zerogeiex,zerogeiez=np.hsplit(zerogeie,[int(len(zerogeie[0])/2)])
            zerogeiex1,zerogeiex2=np.hsplit(zerogeiex,[int(len(zerogeiex[0])/2)])
            zerogeiez1,zerogeiez2=np.hsplit(zerogeiez,[int(len(zerogeiez[0])/2)])
            ss=[]
            ss.append(invtrunmap(zerogeiex1[k],m)+x[1][i][0][0])
            ss.append(invtrunmap(zerogeiex2[k],m)+x[1][i][1][0])
            ss.append(invtrunmap(zerogeiez1[k],m)+x[1][i][2][0])
            ss.append(invtrunmap(zerogeiez2[k],m)+x[1][i][3][0])
            s.append(ss)
        exc=[]
        for i in range(len(s)):
            excc=[]
            for j in range(len(s[i])):
                excc.append([s[i][j]])
            exc.append(excc)
        exc=np.array(exc)
        return exc
def spin(*A,m,N,q):
    x=anyon(*A,m=m,N=N,va='x')
    y=anyon(*A,m=m,N=N,va='y')
    ystr=stringoper(*A,m=m,N=N,va='y')
    nx=x[2]
    ny=y[2]
    coffu1=f2Poly()
    coffu2=f2Poly()
    coffu3=f2Poly()
    for i in range(q):
        coffu1=coffu1+f2Poly({(-(i+1)*nx,0)})
    for i in range(q+1):
        coffu2=coffu2+f2Poly({(0,-i*ny)})
        coffu3=coffu3+f2Poly({(i*nx,0)})
    u1=[]
    u2=[]
    u3=[]
    for i in range(len(x[1])):
        u11=[]
        u22=[]
        u33=[]
        for j in range(len(x[1][i])):
            u11.append([coffu1*x[1][i][j][0]])
            u22.append([coffu2*ystr[i][j][0]])
            u33.append([coffu3*x[1][i][j][0]])
        u1.append(u11)
        u2.append(u22)
        u3.append(u33)
    the=[]
    for i in range(len(u1)):
        coffe=0
        if (0,0) in commute(u1[i],u2[i]).terms:
            coffe=coffe+1
        if (0,0) in commute(u2[i],u3[i]).terms:
            coffe=coffe+1
        if (0,0) in commute(u3[i],u1[i]).terms:
            coffe=coffe+1
        the.append(np.exp(coffe*1j*np.pi))
    u1=[]
    u2=[]
    u3=[]
    for i in range(len(x[1])-1):
        u10=[]
        u20=[]
        u30=[]
        for j in range(i+1,len(x[1])):
            u11=[]
            u22=[]
            u33=[]
            for k in range(len(x[1][i])):
                u11.append([coffu1*(x[1][i][k][0]+x[1][j][k][0])])
                u22.append([coffu2*(ystr[i][k][0]+ystr[j][k][0])])
                u33.append([coffu3*(x[1][i][k][0]+x[1][j][k][0])])
            u10.append(u11)
            u20.append(u22)
            u30.append(u33)
        u1.append(u10)
        u2.append(u20)
        u3.append(u30)
    theta=[]
    for i in range(len(u1)):
        thetai=[]
        for j in range(len(u1[i])):
            coffe=0
            if (0,0) in commute(u1[i][j],u2[i][j]).terms:
                coffe=coffe+1
            if (0,0) in commute(u2[i][j],u3[i][j]).terms:
                coffe=coffe+1
            if (0,0) in commute(u3[i][j],u1[i][j]).terms:
                coffe=coffe+1
            thetai.append(np.exp(coffe*1j*np.pi))
        theta.append(thetai)
    return np.array(the),theta
x1=[[f2Poly({(0,0):1})],
    [f2Poly()],
    [f2Poly()],
    [f2Poly()]]
x2=[[f2Poly()],
    [f2Poly({(0,0):1})],
    [f2Poly()],
    [f2Poly()]]
z1=[[f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0):1})],
    [f2Poly()]]
z2=[[f2Poly()],
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0):1})]]
s1=[
    [f2Poly({(0, 0),(-2,0)})],
    [f2Poly({(0, 0),(0,-1)})],
    [f2Poly()],
    [f2Poly()]
]
s2=[
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0, 0),(0,1)})],
    [f2Poly({(0, 0),(2,0)})],
]
x=anyon(s1,s2,m=2,N=5,va='x')

In [129]:
sp=spin(s1,s2,m=2,N=5,q=10)

In [131]:
print(sp)

(array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j]), [[(1+0j), (-1+1.2246467991473532e-16j), (1+0j)], [(1+0j), (-1+1.2246467991473532e-16j)], [(1+0j)]])


In [122]:
sesp=[]
any=[]
n=len(sp[0])
for i in range(len(sp[0])):
    if sp[0][i].real==1:
        sesp.append(i)
        break
aany=[]
aany.append(x[0][sesp[0]])
t=0
for i in range(0,sesp[0]):
    t=t+n-i-1
tt=i+1
for i in range(t,len(sp[1])):
    if sp[1][i].real==-1:
        tt=tt+i-t
        break
aany.append(x[0][tt])
any.append(np.array(aany))
del x[0][sesp[0]]
del x[0][tt]
del x[1][sesp[0]]
del x[1][tt]
del sp[0][sesp[0]]
del sp[0][tt]
print(any)
print(x[0])
print(sp[0])
print(sp[1])

[array([[0, x^-2y^-2],
       [x^-2y^-2, 0]], dtype=object)]
[[0 x^-2y^-2]
 [0 x^-1y^-2]
 [x^-2y^-2 0]
 [x^-1y^-2 0]]
[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[ 1.+0.0000000e+00j -1.+1.2246468e-16j  1.+0.0000000e+00j
  1.+0.0000000e+00j -1.+1.2246468e-16j  1.+0.0000000e+00j]


In [77]:
musp=[]
if sesp==[]:
    for i in range(len(sp[1])):
        if sp[1][i].real==-1:
            musp.append(i+2)
            break
print(sesp)
print(musp)

(array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j]), array([ 1.+0.0000000e+00j, -1.+1.2246468e-16j,  1.+0.0000000e+00j,
        1.+0.0000000e+00j, -1.+1.2246468e-16j,  1.+0.0000000e+00j]))
